In [10]:
import os, sys, glob

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import loompy
import logging

logger = logging.getLogger()
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%H:%M:%S')

In [5]:
samples = np.loadtxt('/home/camiel/ID_samples.txt')

In [34]:
def get_metadata(cell_ranger, sample):
    dirs = glob.glob(f'{cell_ranger}/{sample}*')
    if len(dirs) > 0:
        dirs = np.array([d for d in dirs if os.path.isdir(d)])
        dirs = dirs[[len(d.split('_')) == 4 for d in dirs]]
        n_flowcells = [len(d.split('_')[-2]) for d in dirs]
        dirs = np.array(dirs)[np.where(n_flowcells==np.max(n_flowcells))[0]]
        ID = [int(d.split('_')[-1]) for d in dirs]

        indir = dirs[np.where(ID==np.max(ID))[0]][0]
        logging.info(f'Using cellranger output: {indir}')
        
#         fs = glob.glob(os.path.join(indir, 'outs/summary*'))
        fs = os.path.join(indir, 'outs/summary.csv')
        return pd.read_csv(fs)
    
X = get_metadata('/data/proj/chromium', f'10X232_1')

14:23:56 INFO     Using cellranger output: /data/proj/chromium/10X232_1_AB_2


In [36]:
X.columns

Index(['Sample ID', 'Genome', 'Pipeline version', 'Estimated number of cells',
       'Confidently mapped read pairs', 'Estimated bulk library complexity',
       'Fraction of genome in peaks',
       'Fraction of high-quality fragments in cells',
       'Fraction of high-quality fragments overlapping TSS',
       'Fraction of high-quality fragments overlapping peaks',
       'Fraction of transposition events in peaks in cells',
       'Fragments flanking a single nucleosome',
       'Fragments in nucleosome-free regions', 'Mean raw read pairs per cell',
       'Median high-quality fragments per cell', 'Non-nuclear read pairs',
       'Number of peaks', 'Percent duplicates', 'Q30 bases in barcode',
       'Q30 bases in read 1', 'Q30 bases in read 2',
       'Q30 bases in sample index i1', 'Sequenced read pairs',
       'Sequencing saturation', 'TSS enrichment score', 'Unmapped read pairs',
       'Valid barcodes'],
      dtype='object')